**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [3]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import random
import pandas as pd

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout
from keras.optimizers import sgd
from keras.layers import Conv2D, Conv2DTranspose, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization, Flatten

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it to maximize a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [4]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [5]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e):
        self.epsilon = e

    def act(self, s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The method **act** determines the next agent's action knowing the current state. When training the RL algorithm(train=True), the agent eather acts according to the best policy it has learned so far or it chooses a random action. The probability of the agent taking a random action during training is ```epsilon```.<br/>
This will allow the agent to better explore the environment and learn new behaviours.<br/>
In other words, **a small/large value of epsilon favors exploitation/exploration**.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [6]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale= 16

        self.to_draw = np.zeros((max_time + 2, grid_size * self.scale, grid_size * self.scale, 3))


    def draw(self, e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x , self.y, :] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :] = 0
        b[:, :2, :] = 0
        
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t, :, :, :] = b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size, 1),
                        self.position.reshape(self.grid_size, self.grid_size, 1)), axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [7]:
# parameters
size = 13
T= 200
temperature= 0.3
epochs_train= 51 # set small when debugging
epochs_test= 11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

```position``` is a 2D array that contains the current position of the rat, set to 1 and all other cells set to 0 except for the boundaries which are set to -1. In this game, the boundaries are the two first and last columns and rows of the array since the rat has a field of view of 2.<br><br>
```board``` is a 2D array that represents the island at a certain timestamp. It contains the cells that have cheese set to 0.5, the cells that have poison set to -1 and all the other cells set to 0 except for the boundaries(set the same way as in position array).<br><br>
If the rat visits a cell, this cell is set to 0 in the ```board``` to show that there is nothing left in this cell.<br><br>
The two arrays have the same shape.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [8]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [9]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        # reset win and lose moves at each start of a new game
        win, lose = 0, 0
        
        # reset environment at the begining of a game and get initial state
        state = env.reset()
        game_over = False
        
        # let agent explore the environment until the game's time limit is reached
        while not game_over:
            # agent acts depending on current state
            action = agent.act(state)
            # apply action and gets next state
            state, reward, game_over = env.act(action)
            
            if reward > 0:
                win += reward
            elif reward < 0:
                lose += reward
        
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score += win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score / (1+e)))
    print('Final score: ' + str(score/epochs))
    return score/epochs

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 8.5/-19.0. Average score (27.5)
Win/lose count 7.5/-10.0. Average score (22.5)
Win/lose count 7.0/-6.0. Average score (19.333333333333332)
Win/lose count 11.0/-18.0. Average score (21.75)
Win/lose count 10.0/-13.0. Average score (22.0)
Win/lose count 6.5/-7.0. Average score (20.583333333333332)
Win/lose count 7.5/-15.0. Average score (20.857142857142858)
Win/lose count 10.5/-8.0. Average score (20.5625)
Win/lose count 10.5/-17.0. Average score (21.333333333333332)
Win/lose count 6.5/-18.0. Average score (21.65)
Win/lose count 10.0/-11.0. Average score (21.59090909090909)
Final score: 21.59090909090909


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, we can show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, we can show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




By minimizing the objective function defined above, we are enforcing the Q-value function to verify the **Bellman** equation(second equation) and thus getting close to the optimal Q-value function.<br>
In other words, if the learned policy converges to the optimal policy, the objective will converge to 0.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [11]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory = self.memory[1:]

    def random_access(self):
        return random.choice(self.memory)

***
The pipeline we will use for training is given below:

In [12]:
def train(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
        
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [13]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1, *s.shape]))[0, :])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            target_q[i] = self.model.predict(s_.reshape([1, *s_.shape]))[0]
            
            if game_over_:
                target_q[i, a_] = r_
            else:
                target_q[i, a_] = r_ + self.discount * max(self.model.predict(n_s_.reshape([1, *n_s_.shape]))[0])
            
            input_states[i] = s_
        # Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential([
          Flatten(input_shape=(5, 5, self.n_state,)),
          Dense(64, input_shape=(5, 5, self.n_state), activation='relu'),
          Dense(128, activation='relu'),
          Dense(64, activation='relu'),
          Dense(4)
        ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/051 | Loss 0.0011 | Win/lose count 4.0/4.0 (0.0)
Epoch 001/051 | Loss 0.0591 | Win/lose count 3.0/5.0 (-2.0)
Epoch 002/051 | Loss 0.0162 | Win/lose count 3.0/5.0 (-2.0)
Epoch 003/051 | Loss 0.0064 | Win/lose count 2.5/4.0 (-1.5)
Epoch 004/051 | Loss 0.0007 | Win/lose count 2.5/3.0 (-0.5)
Epoch 005/051 | Loss 0.0079 | Win/lose count 2.0/5.0 (-3.0)
Epoch 006/051 | Loss 0.0089 | Win/lose count 1.0/2.0 (-1.0)
Epoch 007/051 | Loss 0.0010 | Win/lose count 0/4.0 (-4.0)
Epoch 008/051 | Loss 0.0049 | Win/lose count 3.0/2.0 (1.0)
Epoch 009/051 | Loss 0.0024 | Win/lose count 4.5/5.0 (-0.5)
Epoch 010/051 | Loss 0.0028 | Win/lose count 4.0/5.0 (-1.0)
Epoch 011/051 | Loss 0.0068 | Win/lose count 4.0/4.0 (0.0)
Epoch 012/051 | Loss 0.0033 | Win/lose count 0.5/3.0 (-2.5)
Epoch 013/051 | Loss 0.0024 | Win/lose count 1.5/7.0 (-5.5)
Epoch 014/051 | Loss 0.0022 | Win/lose count 3.5/6.0 (-2.5)
Epoch 015/051 | Loss 0.0067 | Win/lose count 4.5/5.0 (-0.5)
Epoch 016/051 | Loss 0.0406 | Win/lose count 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [15]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        # CNN Model
        model = Sequential([
            Conv2D(32, 2, activation='relu'),
            Conv2D(64, 2, activation='relu'),
            Flatten(),
            Dense(128),
            Dense(4)
        ])
        
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/051 | Loss 0.0045 | Win/lose count 5.5/4.0 (1.5)
Epoch 001/051 | Loss 0.0058 | Win/lose count 6.0/5.0 (1.0)
Epoch 002/051 | Loss 0.0473 | Win/lose count 2.5/0 (2.5)
Epoch 003/051 | Loss 0.0033 | Win/lose count 4.5/1.0 (3.5)
Epoch 004/051 | Loss 0.0062 | Win/lose count 3.5/3.0 (0.5)
Epoch 005/051 | Loss 0.0017 | Win/lose count 5.0/3.0 (2.0)
Epoch 006/051 | Loss 0.0012 | Win/lose count 3.0/4.0 (-1.0)
Epoch 007/051 | Loss 0.0016 | Win/lose count 2.0/2.0 (0.0)
Epoch 008/051 | Loss 0.0036 | Win/lose count 4.0/1.0 (3.0)
Epoch 009/051 | Loss 0.0048 | Win/lose count 2.5/1.0 (1.5)
Epoch 010/051 | Loss 0.0021 | Win/lose count 0.5/1.0 (-0.5)
Epoch 011/051 | Loss 0.0012 | Win/lose count 5.0/2.0 (3.0)
Epoch 012/051 | Loss 0.0530 | Win/lose count 6.0/1.0 (5.0)
Epoch 013/051 | Loss 0.0024 | Win/lose count 12.5/2.0 (10.5)
Epoch 014/051 | Loss 0.0024 | Win/lose count 4.5/2.0 (2.5)
Epoch 015/051 | Loss 0.0525 | Win/lose count 11.0/2.0 (9.0)
Epoch 016/051 | Loss 0.0005 | Win/lose count 7.5/3.0 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
# observe final average score(over number of games) for different temperatures(proba distr of bonus and malus)
scores_df = pd.DataFrame(columns=['temperature', 'cnn_avg_score', 'fc_avg_score'])

for tempr in np.arange(.1, 1, 0.1):
    print(f"Testing for temperature: {tempr}")
    
    env = Environment(grid_size=size, max_time=T,temperature=tempr)
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='cnn_trainmodel.h5', name_model='cnn_trainmodel.json')

    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='fc_trainmodel.h5', name_model='fc_trainmodel.json')
    print('Test of the CNN')
    cnn_score = test(agent_cnn, env, epochs_test, prefix='cnn_test')
    print('Test of the FC')
    fc_score = test(agent_fc, env, epochs_test, prefix='fc_test')
    print('\n')
    scores_df = scores_df.append({'temperature': tempr, 'cnn_avg_score': cnn_score, 'fc_avg_score': fc_score}, 
                                 ignore_index=True)
    
print(scores_df)

Testing for temperature: 0.1
Test of the CNN
Win/lose count 2.0/0. Average score (2.0)
Win/lose count 2.5/-2.0. Average score (3.25)
Win/lose count 1.0/-1.0. Average score (2.8333333333333335)
Win/lose count 4.5/-1.0. Average score (3.5)
Win/lose count 2.0/-2.0. Average score (3.6)
Win/lose count 4.5/-2.0. Average score (4.083333333333333)
Win/lose count 2.0/0. Average score (3.7857142857142856)
Win/lose count 4.0/0. Average score (3.8125)
Win/lose count 5.0/-1.0. Average score (4.055555555555555)
Win/lose count 2.5/-2.0. Average score (4.1)
Win/lose count 4.0/-1.0. Average score (4.181818181818182)
Final score: 4.181818181818182
Test of the FC
Win/lose count 1.5/-5.0. Average score (6.5)
Win/lose count 0.5/-1.0. Average score (4.0)
Win/lose count 0.5/-1.0. Average score (3.1666666666666665)
Win/lose count 0/-2.0. Average score (2.875)
Win/lose count 1.0/0. Average score (2.5)
Win/lose count 2.0/-1.0. Average score (2.5833333333333335)
Win/lose count 0/-1.0. Average score (2.3571428571

Win/lose count 3.5/-3.0. Average score (11.9)
Win/lose count 14.0/-5.0. Average score (13.083333333333334)
Win/lose count 14.0/-8.0. Average score (14.357142857142858)
Win/lose count 7.5/-4.0. Average score (14.0)
Win/lose count 5.0/-2.0. Average score (13.222222222222221)
Win/lose count 15.5/-10.0. Average score (14.45)
Win/lose count 4.0/-5.0. Average score (13.954545454545455)
Final score: 13.954545454545455


Testing for temperature: 0.8
Test of the CNN
Win/lose count 30.5/-1.0. Average score (31.5)
Win/lose count 10.5/-1.0. Average score (21.5)
Win/lose count 32.0/-1.0. Average score (25.333333333333332)
Win/lose count 22.0/-4.0. Average score (25.5)
Win/lose count 27.5/-1.0. Average score (26.1)
Win/lose count 22.5/-4.0. Average score (26.166666666666668)
Win/lose count 44.5/-4.0. Average score (29.357142857142858)
Win/lose count 31.5/-4.0. Average score (30.125)
Win/lose count 38.5/-2.0. Average score (31.27777777777778)
Win/lose count 25.0/-1.0. Average score (30.75)
Win/lose c

In [18]:
HTML(display_videos('cnn_test10.mp4'))

In [19]:
HTML(display_videos('fc_test10.mp4'))

We can notice two things:
<ul>
    <li>In average, the CNN model outperforms the FFN model</li>
        <li>The score is higher when the temperature rises, which is expected because the temperature represents           how likely the game board contains bonus cells</li> 
</ul>

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [20]:
def train_explore(agent, env, epoch, eps_decay=0.1, prefix=''):
    # same as previous train function except that after each game move, we apply a decay on the exploration 
    # parameter epsilon
    
    # Number of won and lost games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # apply eps decay
            agent.set_epsilon(agent.epsilon * (1 - eps_decay))
            
            # The agent performs an action
            action = agent.act(state)
        
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [21]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale= 16

        self.to_draw = np.zeros((max_time + 2, grid_size * self.scale, grid_size * self.scale, 3))


    def draw(self, e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x , self.y, :] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :] = 0
        b[:, :2, :] = 0
        
        b = cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t, :, :, :] = b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward += self.visited_positions[self.x, self.y]
        self.visited_positions[self.x, self.y] = -0.1
        reward += self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1),
                                self.visited_positions.reshape(self.grid_size, self.grid_size, 1)), axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        # initialize state of visited cells to 0
        self.visited_positions = np.zeros((self.grid_size, self.grid_size))
        
        # Now the state contains anoter layer that represents the visited cells
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1),
                                self.visited_positions.reshape(self.grid_size, self.grid_size, 1)), axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [22]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
# we start with a large epsilon value (1 for example which means the first action of the rat is random)
agent = DQN_CNN(size, lr=.1, epsilon = 1, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/051 | Loss 0.0071 | Win/lose count 7.5/20.100000000000026 (-12.600000000000026)
Epoch 001/051 | Loss 0.0040 | Win/lose count 5.5/19.1 (-13.600000000000001)
Epoch 002/051 | Loss 0.0062 | Win/lose count 8.5/17.69999999999998 (-9.199999999999982)
Epoch 003/051 | Loss 0.0030 | Win/lose count 9.5/16.999999999999975 (-7.499999999999975)
Epoch 004/051 | Loss 0.0045 | Win/lose count 3.0/19.200000000000003 (-16.200000000000003)
Epoch 005/051 | Loss 0.0070 | Win/lose count 9.0/18.099999999999994 (-9.099999999999994)
Epoch 006/051 | Loss 0.0058 | Win/lose count 8.0/17.199999999999974 (-9.199999999999974)
Epoch 007/051 | Loss 0.0053 | Win/lose count 18.5/12.599999999999971 (5.900000000000029)
Epoch 008/051 | Loss 0.0075 | Win/lose count 26.0/13.099999999999978 (12.900000000000022)
Epoch 009/051 | Loss 0.0072 | Win/lose count 10.0/16.19999999999996 (-6.19999999999996)
Epoch 010/051 | Loss 0.0066 | Win/lose count 7.0/17.59999999999998 (-10.59999999999998)
Epoch 011/051 | Loss 0.0035 | Win/

In [23]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 9.5/-2.0. Average score (11.5)
Win/lose count 22.5/-2.0. Average score (18.0)
Win/lose count 19.0/-1.0. Average score (18.666666666666668)
Win/lose count 11.5/0. Average score (16.875)
Win/lose count 29.0/-4.0. Average score (20.1)
Win/lose count 16.5/-1.0. Average score (19.666666666666668)
Win/lose count 21.5/-1.0. Average score (20.071428571428573)
Win/lose count 20.0/-1.0. Average score (20.1875)
Win/lose count 26.5/-2.0. Average score (21.11111111111111)
Win/lose count 20.5/0. Average score (21.05)
Win/lose count 12.5/-1.0. Average score (20.363636363636363)
Final score: 20.363636363636363


***